<a href="https://colab.research.google.com/github/gustavoamora/patter_recognition_and_machine_learningning/blob/main/trabalho3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Teste 1:

In [ ]:
# Importando bibliotecas
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

In [ ]:
### Testando primeira base de dados:

from sklearn.datasets import fetch_olivetti_faces
X, y = fetch_olivetti_faces(return_X_y=True)
X.shape, y.shape

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


((400, 4096), (400,))

In [ ]:
from sklearn.model_selection import train_test_split

Xtr, Xte, ytr, yte = train_test_split(X, y)
Xtr.shape, ytr.shape, Xte.shape, yte.shape

((300, 4096), (300,), (100, 4096), (100,))

In [ ]:
# Pipeline Sem GridSearch

pipe_sem_gs = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

score_sem_gs = cross_validate(pipe_sem_gs, X, y,
                        scoring=make_scorer(mean_squared_error, squared=False))
rmse = np.mean(score_sem_gs['test_score'])
std = np.std(score_sem_gs['test_score'])
print(f"KNNstd: {rmse}, std: {std}")

KNNstd: 7.701875299801463, std: 0.4986149479389173


In [ ]:
# Com GridSearch dentro do pipe

parametros = {'n_neighbors': [1, 3, 5, 7, 9, 15]}
scaler = StandardScaler()
GridSearchKNN = GridSearchCV(KNeighborsClassifier(), parametros, scoring='neg_mean_squared_error')

modelo = Pipeline([
    ('padronização', scaler),
    ('gsknn', GridSearchKNN)
])

scores_gs_pipe = cross_validate(modelo, X, y,
                                scoring=make_scorer(mean_squared_error, greater_is_better=False, squared = False))
print(scores_gs_pipe['test_score'])
gs_pipe  = np.mean(scores_gs_pipe['test_score'])
print(f'GS -> Pipe: {gs_pipe}')

[-4.99874984 -4.97619332 -4.56891672 -4.42012443 -6.69794745]
GS -> Pipe: -5.132386353596913


In [ ]:
# Com Pipeline dentro do GridSearch

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])
param = {'scaler__with_mean': [True, False],
         'clf__n_neighbors': [1, 3, 5, 7, 9, 15],
         'clf__metric': ['cosine', 'cityblock', 'euclidean']}
modeloAninhado = GridSearchCV(pipe, param, verbose=0, cv=5,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))
score_pipe_gs = cross_validate(modeloAninhado, Xtr, ytr, cv=5, return_estimator=True,
                        scoring=make_scorer(mean_squared_error, squared=False))
print(score_pipe_gs['test_score'])
pipe_gs  = np.mean(score_pipe_gs['test_score'])
print(f'Pipe -> GS: {pipe_gs}')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

[5.5617743  5.6862407  6.15765107 1.88414437 7.29611769]
Pipe -> GS: 5.317185624972493


In [ ]:
std_diff = round((rmse - pipe_gs) / std,2)
print(f'A diferença de score do modelo com Pipeline dentro do Gridsearch foi de {std_diff}x o std do modelo padrão sem GridSearch.')

A diferença de score do modelo com Pipeline dentro do Gridsearch foi de 6.38x o std do modelo padrão sem GridSearch.
